<a href="https://colab.research.google.com/github/123nol/SSM/blob/main/new_climate_ssm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adding the data (opt 2)



In [ ]:
from google.colab import files

# Upload kaggle.json
files.upload()

# Move kaggle.json to the correct location and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d sumanthvrao/daily-climate-time-series-data
!unzip -o daily-climate-time-series-data.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data
License(s): CC0-1.0
Archive:  daily-climate-time-series-data.zip
  inflating: DailyDelhiClimateTest.csv  
  inflating: DailyDelhiClimateTrain.csv  


In [ ]:
import pandas as pd
df = pd.read_csv('DailyDelhiClimateTrain.csv')

# df=df.fillna(df.mean())

In [ ]:
# Check for missing values
missing_summary = df.isnull().sum()
print("Missing values per column:\n", missing_summary)

# For demonstration, fill missing values with column mean (for numeric columns)
numeric_cols = df.select_dtypes(include='number').columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Drop rows with any remaining missing values (e.g., in non-numeric columns)
df = df.dropna()
print("Remaining missing values:", df.isnull().sum().sum())

Missing values per column:
 date            0
meantemp        0
humidity        0
wind_speed      0
meanpressure    0
dtype: int64
Remaining missing values: 0


In [ ]:
from sklearn.preprocessing import StandardScaler

# Select features for prediction (example: Temperature, Humidity, WindSpeed, Pressure)
feature_cols = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']
target_col = 'meantemp'

scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

df[feature_cols].head()

,meantemp,humidity,wind_speed,meanpressure
0,-2.109500,1.415439,-1.491699,0.025321
1,-2.463454,1.862828,-0.838196,0.037162
2,-2.495219,1.564569,-0.475626,0.041972
3,-2.291015,0.630022,-1.221233,0.033647
4,-2.654044,1.554627,-0.680303,0.029946


In [ ]:
import numpy as np

sequence_length = 7  # Use past 7 days to predict next day

def create_sequences(data, feature_cols, target_col, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[feature_cols].iloc[i:i+seq_len].values)
        y.append(data[target_col].iloc[i+seq_len])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(df, feature_cols, target_col, sequence_length)
print("Input shape:", X_train.shape)
print("Target shape:", y_train.shape)

Input shape: (1455, 7, 4)
Target shape: (1455,)


In [ ]:

test_df = pd.read_csv('DailyDelhiClimateTest.csv')

# Preprocess test data: handle missing values as done for train
test_df[numeric_cols] = test_df[numeric_cols].fillna(df[numeric_cols].mean())
test_df = test_df.dropna()

# Normalize test features using the scaler fitted on train data
test_df[feature_cols] = scaler.transform(test_df[feature_cols])

# Create input sequences and targets for test data
X_test, y_test = create_sequences(test_df, feature_cols, target_col, sequence_length)



In [ ]:
import torch
import torch.nn as nn

class StateSpaceModel(nn.Module):
    def __init__(self, input_dim, state_dim):
        super(StateSpaceModel, self).__init__()
        self.state_dim = state_dim
        self.input_dim = input_dim

        # State update: x_{t+1} = A x_t + B u_t
        self.A = nn.Parameter(torch.eye(state_dim))
        self.B = nn.Parameter(torch.randn(state_dim, input_dim))

        # Output: y_t = C x_t + D u_t
        self.C = nn.Parameter(torch.randn(1, state_dim))
        self.D = nn.Parameter(torch.randn(1, input_dim))

    def forward(self, u_seq):
        # u_seq: (batch, seq_len, input_dim)
        batch_size, seq_len, _ = u_seq.size()
        x = torch.zeros(batch_size, self.state_dim, device=u_seq.device)
        for t in range(seq_len):
            u_t = u_seq[:, t, :]
            x = torch.matmul(x, self.A.T) + torch.matmul(u_t, self.B.T)
        # Output for the last state/input
        y = torch.matmul(x, self.C.T) + torch.matmul(u_t, self.D.T)
        return y.squeeze(1)

# Example instantiation
input_dim = len(feature_cols)
state_dim = 8
model = StateSpaceModel(input_dim, state_dim)
print(model)

StateSpaceModel()


In [ ]:
import torch.optim as optim

# Convert data to PyTorch tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_torch = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_torch = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).to(device)

model = StateSpaceModel(input_dim, state_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_torch)
    loss = criterion(output, y_train_torch)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 5/100, Loss: 43.4796
Epoch 10/100, Loss: 20.3033
Epoch 15/100, Loss: 6.8000
Epoch 20/100, Loss: 7.2733
Epoch 25/100, Loss: 2.6920
Epoch 30/100, Loss: 2.9240
Epoch 35/100, Loss: 1.3225
Epoch 40/100, Loss: 1.2963
Epoch 45/100, Loss: 0.7059
Epoch 50/100, Loss: 0.6194
Epoch 55/100, Loss: 0.4211
Epoch 60/100, Loss: 0.3608
Epoch 65/100, Loss: 0.2964
Epoch 70/100, Loss: 0.2724
Epoch 75/100, Loss: 0.2463
Epoch 80/100, Loss: 0.2351
Epoch 85/100, Loss: 0.2199
Epoch 90/100, Loss: 0.2123
Epoch 95/100, Loss: 0.2014
Epoch 100/100, Loss: 0.1947


In [ ]:
# import torch.nn.functional as F

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # x: (batch, seq_len, input_dim)
        out, _ = self.lstm(x)
        # Use the last output for prediction
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze(1)

# Hyperparameters for LSTM
hidden_dim = 16
num_layers = 1

lstm_model = LSTMModel(input_dim, hidden_dim, num_layers).to(device)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=0.01)
lstm_criterion = nn.MSELoss()

# Training loop for LSTM
epochs = 30
for epoch in range(epochs):
    lstm_model.train()
    lstm_optimizer.zero_grad()
    output = lstm_model(X_train_torch)
    loss = lstm_criterion(output, y_train_torch)
    loss.backward()
    lstm_optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"[LSTM] Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


[LSTM] Epoch 5/30, Loss: 0.7498
[LSTM] Epoch 10/30, Loss: 0.3500
[LSTM] Epoch 15/30, Loss: 0.1963
[LSTM] Epoch 20/30, Loss: 0.1468
[LSTM] Epoch 25/30, Loss: 0.1076
[LSTM] Epoch 30/30, Loss: 0.1070


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

model.eval()
with torch.no_grad():
    y_pred = model(X_test_torch).cpu().numpy()
    y_true = y_test_torch.cpu().numpy()

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

Test MAE: 0.3381
Test RMSE: 0.4175


In [ ]:
lstm_model.eval()
with torch.no_grad():
    y_pred_lstm = lstm_model(X_test_torch).cpu().numpy()
    y_true_lstm = y_test_torch.cpu().numpy()

lstm_mae = mean_absolute_error(y_true_lstm, y_pred_lstm)
lstm_rmse = np.sqrt(mean_squared_error(y_true_lstm, y_pred_lstm))

print(f"LSTM Test MAE: {lstm_mae:.4f}")
print(f"LSTM Test RMSE: {lstm_rmse:.4f}")

LSTM Test MAE: 0.2551
LSTM Test RMSE: 0.3042
